In [4]:
import sys
sys.path.append("../")
print(sys.path)

['/home/ubu/anaconda3/envs/recoedge/lib/python3.8/site-packages/ray/thirdparty_files', '/home/ubu/anaconda3/envs/recoedge/lib/python3.8/site-packages/ray/pickle5_files', '/home/ubu/RecoEdge/notebooks', '/home/ubu/.vscode-server/extensions/ms-toolsai.jupyter-2021.10.1101450599/pythonFiles/vscode_datascience_helpers', '/home/ubu/.vscode-server/extensions/ms-toolsai.jupyter-2021.10.1101450599/pythonFiles', '/home/ubu/.vscode-server/extensions/ms-toolsai.jupyter-2021.10.1101450599/pythonFiles/lib/python', '/home/ubu/anaconda3/envs/recoedge/lib/python38.zip', '/home/ubu/anaconda3/envs/recoedge/lib/python3.8', '/home/ubu/anaconda3/envs/recoedge/lib/python3.8/lib-dynload', '', '/home/ubu/anaconda3/envs/recoedge/lib/python3.8/site-packages', '/home/ubu/anaconda3/envs/recoedge/lib/python3.8/site-packages/IPython/extensions', '/home/ubu/.ipython', '../', '../']


In [5]:
import yaml
import json
from json import dumps, loads
from kafka import KafkaProducer, KafkaConsumer
from fedrec.communications.messages import JobSubmitMessage
from fedrec.utilities import registry

with open("../configs/dlrm_fl.yml", 'r') as cfg:
    config = yaml.load(cfg, Loader=yaml.FullLoader)

def init_kafka(config):
    producer_url = "{}:{}".format(
        config["producer_url"], config["producer_port"])
    return KafkaProducer(
        bootstrap_servers=[producer_url])

serializer = registry.construct("serializer", "json")
# config = config["multiprocessing"]["communications"]

producer = init_kafka(config["multiprocessing"]["communications"])
producer.send('testing1', value=serializer.serialize(JobSubmitMessage("test_run",[1,2],{},"id1","id2",None)))
# producer.send('testing1', value={"test": "test_abhinav"})

{'__type__': 'JobSubmitMessage', '__data__': {'senderid': 'id1', 'receiverid': 'id2', 'job_type': 'test_run', 'job_args': [1, 2], 'job_kwargs': {}, 'workerstate': None}} <class 'dict'>


In [6]:
with open("../configs/dlrm_fl.yml", 'r') as cfg:
    config = yaml.load(cfg, Loader=yaml.FullLoader)

ag_config = {
        # Seed for RNG used in shuffling the training data.
    "data_seed" : 100,
    # Seed for RNG used in initializing the model.
    "init_seed" : 100,
    # Seed for RNG used in computing the model's training loss.
    # Only relevant with internal randomness in the model, e.g. with dropout.
    "model_seed" : 100
}



In [7]:
producer = init_kafka(config["multiprocessing"]["communications"])

In [8]:
producer.send('testing1', value={"test": "test_abhinav"})

AssertionError: 

In [ ]:
aggregator_cfg = AggregatorConfig(**ag_config)
JHhook = ray.remote(JobExecutor)
je_Aggregator = JHhook.remote(FedAvg,
                                config,
                                aggregator_cfg,
                                NoOpLogger())
je_Aggregator.run.remote()

In [ ]:
ray.shutdown()


In [ ]:
rpm = RayProcessManager()
rpm.distribute(JobExecutor, FedAvg.__name__, 1 ,
                        FedAvg, config["model"], aggregator_cfg, NoOpLogger())

rpm.start(FedAvg.__name__, "run")